# 计算非灵活用水的贡献

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

import pandas as pd
import numpy as np

In [ ]:
from hydra import compose, initialize
import os

# 加载项目层面的配置
with initialize(version_base=None, config_path="../config"):
    cfg = compose(config_name="config")
os.chdir(cfg.root)

colors = cfg.style.colors

In [ ]:
from regimes_yrb.tools.statistic import (
    ratio_contribution,
    plot_pittitt_change_points,
    plot_ratio_contribution,
    pettitt_change_points,
)
import matplotlib
from matplotlib import pyplot as plt
from matplotlib.gridspec import GridSpec

In [ ]:
# 加载阈值为 0.05的数据，即与黄河流域相交面积大于全市总面积 5% 的所有市
city_yr = pd.read_csv(cfg.db.perfectures)

### 提取非灵活用水

非灵活用水包括：

1. 城市居民用水
2. 农村居民用水
3. 农村牲畜用水
4. 火电冷却用水

In [ ]:
## 提取非灵活用水的数据

# 与基本信息有关的列
use_cols = ["Year", "Region", "City_ID"]

# 与耗水量有关的列
wu_cols = [
    "Total water use",
    "IRR",
    "IND",
    "RUR",
    "URB",
    "Urban domestic WU",
    "Rural domestic WU",
    "Rural livestock WU",
    "Urban service WU",
    "Industrial gross value added (GVA): Thermal electrivity",
    "Industrial WUI: Thermal electrivity",
]

use_cols.extend(wu_cols)  # 在计算SFV中所有使用的数据列

# 加载所有使用的数据
consumptions = city_yr[use_cols].copy()

# 计算火电冷却用水的量
consumptions["Thermal WU"] = (
    consumptions["Industrial gross value added (GVA): Thermal electrivity"]
    * consumptions["Industrial WUI: Thermal electrivity"]
)
consumptions.drop(wu_cols[-2:], axis=1, inplace=True)  # 删除计算后的 WUI, GVA 数据

for col in consumptions.columns:
    if col in wu_cols or col in cfg.inflexible_wu:
        consumptions[col] = consumptions[col] * 10  # 转化单位 km3 → 10^8 m3

# 展示数据
consumptions.head()
# 导出数据
consumptions.to_csv(cfg.db.consumptions)

## 非灵活用水占比

In [ ]:
inflexible_water_use = consumptions.groupby(["Year"]).sum(numeric_only=True)[
    cfg.inflexible_wu
]
inflexible_water_use_sum = inflexible_water_use.sum(axis=1, numeric_only=True)
total_water_use = consumptions.groupby(["Year"]).sum(numeric_only=True)[
    "Total water use"
]

# 非灵活用水占比
inflexible_ratio = inflexible_water_use_sum / total_water_use

# 非灵活用水贡献
inflexible_contributions = ratio_contribution(
    numerator=inflexible_water_use, denominator=total_water_use
)

## 作图

In [ ]:
# 绘图
fig = plt.figure(figsize=(8, 3.25), constrained_layout=True)
gs = GridSpec(1, 2, figure=fig, width_ratios=[3, 4])
ax1 = fig.add_subplot(gs[0, 0])
ax2 = fig.add_subplot(gs[0, 1])

# 绘制图片 1
inflexible_slopes = plot_pittitt_change_points(
    inflexible_ratio, change_points=[1977, 1993], colors=colors.period, ax=ax1
)

# 绘制图片 2
plot_ratio_contribution(
    inflexible_contributions,
    denominator_color=colors.total_WU,
    denominator_label="Total water use",
    ax=ax2,
)

# 修饰图片 1
ax1.set_xlabel("Year")
ax1.set_ylabel("Proportion of inflexible water use")
ax1.set_yticks(np.arange(0.04, 0.20, 0.05))
ax1.legend(loc=2)
ax1.text(2010, 0.05, "a.", ha="center", va="center", weight="bold", size="large")

# 修饰图片 2
ax2.set_xlabel("Different periods")
ax2.set_ylabel("Changes of inflexible water use proportion")
ax2.axvline(1.5, ls=":", color="gray", lw=1.5)
ax2.axvline(0.5, ls=":", color="gray", lw=1.5)
ax2.axhline(0.0, lw=2, color="gray")
ax2.set_yticks(np.arange(-0.05, 0.11, 0.05))
ax2.set_ylim(-0.05, 0.11)
ax2.text(2.4, -0.04, "b.", ha="center", va="center", weight="bold", size="large")
ax2.tick_params(axis="x", tickdir="in", bottom=False, labelrotation=0)

# 调整坐标轴显示
for ax in [ax1, ax2]:
    ax.spines["top"].set_visible(False)
    ax.spines["bottom"].set_visible(True)
    ax.spines["left"].set_visible(True)
    ax.spines["right"].set_visible(False)
ax2.spines["bottom"].set_visible(False)

# 出图
# plt.savefig('../figures/sup/inflexible_wu.pdf', dpi=300)
# plt.savefig('../figures/sup/inflexible_wu.jpg', dpi=300)
plt.show();